# Getting Started with Taipy GUI

!!! important "Supported Python versions"

    Taipy requires **Python 3.8** or newer.

Welcome to the **Getting Started** guide for Taipy GUI. This tour shows you how to use Taipy GUI to create an interactive Web application. Taipy GUI implements a modern backend for any data-driven application based on your business case.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_00/imd_end_interface.png width=700>
</div>

# Taipy GUI

Taipy GUI is one of the components of Taipy to create easily Web application. There are a lot of reasons for using Taipy GUI:

- It fastens the creation of an application.

- It manages easily and efficiently variables and events.

- Easy visualization with Markdown syntax.

Each step of the **"Getting Started"** will focus on basic concepts of *Taipy*. Note that every step is dependent on 
the code of the previous one. After completing the last step, you will have the skills to develop your own Taipy 
application. 

## Before we begin

Only Taipy has to be installed. **Taipy** package requires Python 3.8 or newer;



In [0]:
# !pip install taipy


## Using Notebooks


> You can download the code for this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_01.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started/tree/develop/src).



# Step 1: First web page

You only need one line of code to create your first Taipy web page. Create a `Gui` object with a String and run it. 
A client link will be displayed in the console. Please enter it in a web browser to open your first Taipy web client!




In [1]:
from taipy import Gui

Gui(page="# Getting started with *Taipy*").run()


 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



If you want to run multiple servers simultaneously, you can change the server port number (5000 by default) in the `.run()` method. For example, `Gui(...).run(port=xxxx)`.


Note that you can style the text. Taipy uses the Markdown syntax to style your text and more. Therefore, `#` creates 
a title, `##` makes a subtitle. Put your text in `*` for *italics* or in `**` to have it in **bold**.


<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_01/result.png width=700>
</div>


> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_02.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).



# Step 2: Visual elements

Many visual elements can be added to the basic code viewed in Step 1. This Step shows how to use visual elements 
like charts, sliders, and tables, and implement them in the GUI.

## Visual elements

Taipy GUI can be considered as an **augmented** Markdown; it adds the concept of 
**[Visual elements](https://docs.taipy.io/en/latest/manuals/gui/viselements/)** on top of all the Markdown syntax. A visual 
element is a Taipy graphical object displayed on the client. It can be a 
[slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/), a 
[chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/), a 
[table](https://docs.taipy.io/en/latest/manuals/gui/viselements/table/), an 
[input](https://docs.taipy.io/en/latest/manuals/gui/viselements/input/), a 
[menu](https://docs.taipy.io/en/latest/manuals/gui/viselements/menu/), etc. Check the list 
[here](https://docs.taipy.io/en/latest/manuals/gui/controls/).

Every visual element follows a similar syntax:

`<|{variable}|visual_element_name|param_1=param_1|param_2=param_2| ... |>`.

For example, a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) is written this way :

`<|{variable}|slider|min=min_value|max=max_value|>`.

For each visual element you wish to add to your web page, you must include the syntax above inside your markdown 
string (representing your page). For example, at the beginning of the page, let's display:

- a Python variable *text*;

- an input that will "visually" modify the value of __text__.

Here is the overall syntax:

```
*<|{text}|>*
<|{text}|input|>
```

Here is the combined code:



In [2]:
from taipy.gui import Gui

text = "Original text"

page = """
# Getting started with Taipy GUI

My text: <|{text}|>

<|{text}|input|>
"""

Gui(page).run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_02/result.png width=700>
</div>


> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_03.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).



# Step 3: Interactive GUI

Now, the page has several visual elements:

- A text that is connected to the Python variable *text*;

- An input that changes the value *text* automatically.

Taipy GUI manages everything. To go further into Taipy GUI, let's consider the concept of **state**.

## Multi-user - state

Try to open a few clients with the same URL. You will see that every client is independent of each other; you can change *text* on a client, and *text* will not change in other clients. This is due to the concept of **state**.

The state holds the value of all the variables used in the user interface for one specific connection.

For example, in the beginning, `state.text = 'Original text'`. When *text* is modified by the input (through a given graphical client), this is, in fact, *state.text* that is modified, not *text* (the global Python variable). Therefore, if you open two different clients, *text* will have two state values (*state.text*), one for each client.

In the code below, this concept will be used to:

- Notify the user when the button is pressed;

- Reset the input when the text equals "Reset".

## How to connect two variables - the *[on_change()](https://docs.taipy.io/en/latest/manuals/gui/callbacks/)* function

In *Taipy*, the `on_change()` function is a "special" function. **Taipy** will check if you created and will use a function with this name. Whenever the state of a variable is modified, the *callback* function is called with three parameters:

- state (the state object containing all the variables);

- The name of the modified variable;

- Its value.

Here, `on_change()` will be called whenever the text's value (*state.text*) changes. If a variable is changed in this function, Taipy will propagate this change automatically to the associated visual elements.

Other callbacks specific to visual elements exist. They are named _on_change_ or _on_action_. For example, a button has an _on_action_ property. When the button is pressed, Taipy will call the function referenced in the _on_action_ property.



In [3]:
from taipy.gui import Gui, notify

text = "Original text"

# Definition of the page
page = """
# Getting started with Taipy GUI

My text: <|{text}|>

<|{text}|input|>

<|Run local|button|on_action=on_button_action|>
"""

def on_button_action(state):
    notify(state, 'info', f'The text is: {state.text}')
    state.text = "Button Pressed"

def on_change(state, var_name, var_value):
    if var_name == "text" and var_value == "Reset":
        state.text = ""
        return


Gui(page).run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_03/result.png width=700>
</div>

_notify()_ is a Taipy GUI function that creates a notification with text. The user can pass multiple parameters, including the _state_, the _notification_type_, and the _message_.


> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_04.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).

# Step 4: Charts
 
Charts are an essential part of Taipy and any Web application. A chart is just another visual element with many properties to customize it.

Here is one of the simplest codes to create a chart:



In [4]:
list_to_display = [100/x for x in range(1, 100)]
Gui("<|{list_to_display}|chart|>").run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



Different formats can be passed to a chart element: a list, a Numpy array, or a Pandas Dataframe.

## Different useful properties

Taipy charts are based on Plotly charts. Like any other visual element, charts have a lot of parameters. Some of them can be indexed to change properties for specific chart traces. 

Here are a few of the essential properties. You can also look at the [documentation]() for more information.
 - x and y are used to define the axis of the chart. Note that even if data inside columns are dynamic, the name of columns to display in a chart are not.



In [7]:
import pandas as pd

data = pd.DataFrame({"x_col":[0,1,2], "y_col1":[4,1,2]})
Gui("<|{data}|chart|x=x_col|y=y_col1|>").run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000



 - x and y can be indexed to add more traces to the chart:



In [8]:
data = pd.DataFrame({"x_col":[0,1,2], "y_col_1":[4,1,2], "y_col_2":[3,1,2]})
Gui("<|{data}|chart|x=x_col|y[1]=y_col_1|y[2]=y_col_2|>").run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



 - Taipy provides a lot of different options to customize graphs. _color_ is one of them:



In [9]:
data = pd.DataFrame({"x_col":[0,1,2], "y_col_1":[4,1,2], "y_col_2":[3,1,2]})
Gui("<|{data}|chart|x=x_col|y[1]=y_col_1|y[2]=y_col_2|type[1]=bar|color[1]=green|>").run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



## Different types of charts

Different types are available: maps, bar charts, pie charts, line charts, and 3D charts, ... To know how to use them quickly, types are listed [here](). If compatible, two types like _scatter_, _line_, and _bar_ on the same chart can also be used together. 



In [8]:
data = pd.DataFrame({"x_col":[0,1,2], "y_col_1":[4,1,2], "y_col_2":[3,1,2]})
"<|{data}|chart|x=x_col|y[1]=y_col_1|y[2]=y_col_2|type[1]=bar|>"



## Code

A chart is added to our code to visualize the score given by our NLP algorithm to different lines.



In [16]:

page = """
... put the previous Markdown page here

<|{dataframe}|table|>

<|{dataframe}|chart|type=bar|x=Text|y[1]=Score Pos|y[2]=Score Neu|y[3]=Score Neg|y[4]=Overall|color[1]=green|color[2]=grey|color[3]=red|type[4]=line|>
"""


dataframe = pd.DataFrame({"Text":['Test', 'Other', 'Love'],
                          "Score Pos":[1, 1, 4],
                          "Score Neu":[2, 3, 1],
                          "Score Neg":[1, 2, 0],
                          "Overall":[0, -1, 4]})
Gui(page).run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:



## Quick tip to write visual elements

To simplify the creation of a visual element. Each of them has a "property" parameter where a Python dictionary of property can be directly passed. To replicate the graph above, we could do the following:



In [17]:
property_chart = {"type":"bar",
                  "x":"Text",
                  "y[1]":"Score Pos",
                  "y[2]":"Score Neu",
                  "y[3]":"Score Neg",
                  "y[4]":"Overall",
                  "color[1]":"green",
                  "color[2]":"grey",
                  "color[3]":"red",
                  "type[4]":"line"
                 }

page = """
...
<|{dataframe}|chart|properties={property_chart}|>
...
"""

Gui(page).run()

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:




<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_04/result.png width=700>
</div>


> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_05.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).

# Step 5: Insert Python expressions in Taipy GUI

As shown before, parameters and variables in Taipy are dynamic. The same follows for every type of object, even data frames. Therefore, you can do operations on data frames, and Taipy GUI will show real-time results on the GUI. These changes occurs through the `=` asignment like `state.xxx = yyy` (`state.text = "Example"`).

Any expression containing `xxx` in the Markdown will propagate the changes and reload related elements. It can be  simple charts or tables, but it can also be an expression like this:



In [19]:
"""
## Positive
<|{np.mean(dataframe['Score Pos'])}|text|>

## Neutral
<|{np.mean(dataframe['Score Neu'])}|text|>

## Negative
<|{np.mean(dataframe['Score Neg'])}|text|>
"""

"\n## Positive\n<|{np.mean(dataframe['Score Pos'])}|text|>\n\n## Neutral\n<|{np.mean(dataframe['Score Neu'])}|text|>\n\n## Negative\n<|{np.mean(dataframe['Score Neg'])}|text|>\n"


This kind of expression creates direct connections between visual elements without coding anything.


## A use case for NLP

The code for NLP is provided here; it doesn't concern Taipy but is helpful for the application.




In [22]:
pip install torch

  Using cached torch-1.13.1-cp310-cp310-win_amd64.whl (162.6 MB)
Note: you may need to restart the kernel to use updated packages.


In [25]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax


MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)


def analyze_text(text):
    # Run for Roberta Model
    encoded_text = tokenizer(text, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    
    return {"Text":text,
            "Score Pos":scores[2],
            "Score Neu":scores[1],
            "Score Neg":scores[0],
            "Overall":scores[2]-scores[0]}



C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:




## Rest of code to implement the NLP element

The code below uses this concept to create metrics on the data frame generated. 




In [26]:
import numpy as np
import pandas as pd 
from taipy.gui import Gui, notify

text = "Original text"

dataframe = pd.DataFrame({"Text":[''],
                          "Score Pos":[0.33],
                          "Score Neu":[0.33],
                          "Score Neg":[0.33],
                          "Overall":[0]})


def local_callback(state):
    notify(state, 'Info', f'The text is: {state.text}', True)
    temp = state.dataframe.copy()
    scores = analize_text(state.text)
    state.dataframe = temp.append(scores, ignore_index=True)
    state.text = ""



page = """
<|toggle|theme|>

# Getting started with Taipy GUI

My text: <|{text}|>

Enter a word:

<|{text}|input|>

<|Analyze|button|on_action=local_callback|>

## Positive
<|{np.mean(dataframe['Score Pos'])}|text|format=%.2f|>

## Neutral
<|{np.mean(dataframe['Score Neu'])}|text|format=%.2f|>

## Negative
<|{np.mean(dataframe['Score Neg'])}|text|format=%.2f|>

<|{dataframe}|table|>

<|{dataframe}|chart|type=bar|x=Text|y[1]=Score Pos|y[2]=Score Neu|y[3]=Score Neg|y[4]=Overall|color[1]=green|color[2]=grey|color[3]=red|type[4]=line|>
"""

Gui(page).run()


Gui server has been stopped.
 * Server starting on http://127.0.0.1:5000


C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\gui.py:902: UserWarning: on_action: Exception raised in function 'local_callback':
name 'analize_text' is not defined
  warnings.warn(f"on_action: Exception raised in function '{action_function.__name__}':\n{e}")
C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\gui.py:873: UserWarning: on_action: 'local_callback' is not a valid function
  warnings.warn(f"on_action: '{action}' is not a valid function")
C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\gui.py:902: UserWarning: on_action: Exception raised in function 'local_callback':
name 'analize_text' is not defined
  warnings.warn(f"on_action: Exception raised in function '{action_function.__name__}':\n{e}")
C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\gui\gui.py:873: UserWarning: on_action: 'local_callback' is not a valid function
  warnings.warn(f"on_action: '{act



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_05/result.png width=700>
</div>


> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_06.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).



# Step 6: Organize the layout of your App

With just a few steps, you have created a full forecasting application that predicts across multiple days with different parameters. However, the page's layout could be greatly improved. To get a more aesthetically pleasing page, three new useful controls will be used. These are:

- [part](https://docs.taipy.io/en/latest/manuals/gui/viselements/part/): creates a group of text/visual elements. A useful property of `part` is _render_. If set to False, it will not display the part. This allows the developer to display a group of visual elements dynamically or not.

```
<|part|render={bool_variable}|
Text
Or visual elements...
|>
```

- [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/): creates invisible columns where you can put your texts and visual elements. The _columns_ property indicates the width and number of columns. Here, we create three columns of the same width.

```
<|layout|columns=1 1 1|
Button in first column <|Press|button|>

Second column

Third column
|>
```

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_06/layout.png width=500>
</div>


- [expendable](https://docs.taipy.io/en/latest/manuals/gui/viselements/expendable/): creates a block that can expand or shrink.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_06/expendable.png width=500>
</div>


## Back to the code



In [14]:

page = """
<|toggle|theme|>

# Getting started with Taipy GUI

<|layout|columns=1 1|
<|
My text: <|{text}|>

Enter a word:

<|{text}|input|>

<|Analyze|button|on_action=local_callback|>
|>


<|Table|expandable|
<|{dataframe}|table|width=100%|>
|>

|>

<|layout|columns=1 1 1|
<|
## Positive
<|{np.mean(dataframe['Score Pos'])}|>
|>

<|
## Neutral
<|{np.mean(dataframe['Score Neu'])}|>
|>

<|
## Negative
<|{np.mean(dataframe['Score Neg'])}|>
|>
|>

<|{dataframe}|chart|type=bar|x=Text|y[1]=Score Pos|y[2]=Score Neu|y[3]=Score Neg|y[4]=Overall|color[1]=green|color[2]=grey|color[3]=red|type[4]=line|>
"""



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_06/result.png width=700>
</div>



> You can download the code of this step [here](https://docs.taipy.io/en/latest/getting_started/src/step_07.py) or all the steps [here](https://github.com/Avaiga/taipy-getting-started-gui/tree/develop/src).

## Step 7: Multi-pages, navbars, and menus

The creation of a multi-page application is greatly simplified through Taipy. To create a multi-page application, a dictionary of pages has to be specified in the definition of the GUI. Pages 1 and 2 will share the content of the root page. Visual elements like a menu or navbar are usually put on this root page to navigate between the different pages of the application.




In [15]:
from taipy import Gui

root_md="# Multi-page application"
page1_md="## This is page 1"
page2_md="## This is page 2"

pages = {
    "/": root_md,
    "page1": page1_md,
    "page2": page2_md
}
Gui(pages=pages).run()



## Navigating between pages

- [menu](https://docs.taipy.io/en/latest/manuals/gui/viselements/menu/): creates a menu on the left to navigate through the pages.

`<|menu|label=Menu|lov={lov_pages}|on_action=on_menu|>`. For example, this code creates a menu with two pages:



In [16]:
from taipy.gui import Gui, navigate


root_md="<|menu|label=Menu|lov={[('Page-1', 'Page 1'), ('Page-2', 'Page 2')]}|on_action=on_menu|>"
page1_md="## This is page 1"
page2_md="## This is page 2"


def on_menu(state, var_name, function_name, info):
    page = info['args'][0]
    navigate(state, to=page)
   
   
pages = {
    "/": root_md,
    "Page-1": page1_md,
    "Page-2": page2_md
}

Gui(pages=pages).run()



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_07/menu.png width=50>
</div>

- [navbar](): creates an element to navigate through the Taipy pages by default



In [17]:
from taipy.gui import Gui


root_md="<|navbar|>"
page1_md="## This is page 1"
page2_md="## This is page 2"

pages = {
    "/": root_md,
    "Page-1": page1_md,
    "Page-2": page2_md
}

Gui(pages=pages).run()


 
<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_07/navbar.png width=50>
</div>

 
## Back to the code

In our application, the first page (named _page_) contains the previous Markdown.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_07/data_visualization.png width=700>
</div>

Then, letâ€™s create our second page, which contains a page to analyze an entire text.



In [18]:
# Second page

dataframe2 = dataframe.copy()
path = ""
treatment = 0

page_file = """
<|{path}|file_selector|extensions=.txt|label=Upload .txt file|on_action=analyze_file|> <|{f'Downloading {treatment}%...'}|>


<|Table|expandable|
<|{dataframe2}|table|width=100%|>
|>

<|{dataframe2}|chart|type=bar|x=Text|y[1]=Score Pos|y[2]=Score Neu|y[3]=Score Neg|y[4]=Overall|color[1]=green|color[2]=grey|color[3]=red|type[4]=line|height=800px|>

"""

def analyze_file(state):
    state.dataframe2 = dataframe2
    state.treatment = 0
    with open(state.path,"r", encoding='utf-8') as f:
        data = f.read()
        # split lines and eliminates duplicates
        file_list = list(dict.fromkeys(data.replace('\n', ' ').split(".")[:-1]))
    
    
    for i in range(len(file_list)):
        text = file_list[i]
        state.treatment = int((i+1)*100/len(file_list))
        temp = state.dataframe2.copy()
        scores = analize_text(text)
        state.dataframe2 = temp.append(scores, ignore_index=True)
        
    state.path = None



This little code below assembles our previous page and this new page. The _navbar_ in the root page is present in both pages and is automatically able to switch pages through it. 



In [19]:

# One root page for common content
# The two pages that were created
pages = {"/":"<|toggle|theme|>\n<center>\n<|navbar|>\n</center>",
         "line":page,
         "text":page_file}


Gui(pages=pages).run()
